<a href="https://colab.research.google.com/github/mathemakitten/fastai-colab/blob/master/lesson1_homework_cats_dogs_redux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install kaggle CLI onto the server instance

2. Use the kaggle CLI to download the current data for the Dogs vs. Cats Redux competition
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition 

3. Configure the new data to the file structure in the same way that was used in the sample lesson 1 notebook

4. Make a copy of the lesson 1 notebook and use the new copy to draw in the new Dogs Vs. Cats data (if you copy the notebook outside of the course folder, don't forget the utils.py, vgg26.py files etc)

5. Run the relevant code cells on the sample set of new Dogs v. Cats data to make a prediction on the new image data set.

6. Once the sample set works, modify the jupyter notebook to use the new test data images

7. Write a script that takes the predict() data of the new Dogs vs. Cats data and writes the data to a new csv file in the format of the sample_submission.csv file that was downloaded with the Dogs vs. Cats

8. Submit that new submission.csv file to the kaggle via the CLI tool

In [1]:
!rm utils.py*
!rm vgg16.py*
!rm vgg16bn.py*

!wget https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/utils.py
!wget https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/vgg16.py
!wget https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/vgg16bn.py
  
! pip install bcolz

from utils import *
from vgg16 import Vgg16

from shutil import copyfile
import zipfile
import os

%matplotlib inline

--2019-02-24 19:35:34--  https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7950 (7.8K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.76K  --.-KB/s    in 0s      

2019-02-24 19:35:34 (118 MB/s) - ‘utils.py’ saved [7950/7950]

--2019-02-24 19:35:35--  https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/vgg16.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9037 (8.8K) [text/plain]
Saving to: ‘vgg16.py’

vgg16.

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ensure that you download your kaggle.json authentication key from the Kaggle website and upload it to your Google Drive.

In [3]:
! cp /content/drive/'My Drive'/'Colab Notebooks'/fastai/kaggle.json /root/.kaggle/kaggle.json
! kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 95% 257M/271M [00:02<00:00, 123MB/s]
100% 271M/271M [00:02<00:00, 131MB/s]
 98% 534M/544M [00:04<00:00, 124MB/s]
100% 544M/544M [00:04<00:00, 136MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 35.4MB/s]


In [4]:
%ls
%mkdir data

data/   __pycache__/  sample_submission.csv  train.zip  vgg16bn.py
drive/  sample_data/  test.zip               utils.py   vgg16.py
mkdir: cannot create directory ‘data’: File exists


In [5]:
MAIN_DIR = os.getcwd()
DATA_DIR = MAIN_DIR + '/data'

% cd $DATA_DIR

/content/data


In [6]:
if (os.path.exists('test') is False) or (os.path.exists('train') is False):
  for file in ("../test.zip", "../train.zip"):
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall('')
    zip_ref.close()
  print("Files unzipped")
else: 
  print("Files already unzipped")

Files unzipped


In [7]:
# Check the files in the directories unzipped properly
!ls train | head -5

cat.0.jpg
cat.10000.jpg
cat.10001.jpg
cat.10002.jpg
cat.10003.jpg


In [8]:
# Determine how many training samples we have - should be 25,000
# A 20% validation set of 25,000 would be 5,000
!ls train | wc -l

25000


In [0]:
# Setup directory structure for data

!mkdir valid
!mkdir results
!mkdir -p sample/train
!mkdir -p sample/valid
!mkdir -p sample/test

In [10]:
# Create a 20% validation set - 5000 images

# Get all the training images
% cd $DATA_DIR
% cd train

imgs = glob('*.jpg')
shuffled = np.random.permutation(imgs)

for i in range(5000): 
  os.rename(shuffled[i], DATA_DIR + '/valid/' + shuffled[i]) # recall that DATA_DIR = /content/data

/content/data
/content/data/train


In [0]:
# Create a sample training dataset by copying over a small 1% sample (200 images)

sample_train_imgs = glob('*.jpg')
shuffled_train_sample = np.random.permutation(sample_train_imgs)

for i in range(200): 
  copyfile(shuffled_train_sample[i], DATA_DIR + '/sample/train/' + shuffled_train_sample[i])

In [12]:
# As above, create a sample validation dataset for 50 validation images 

% cd $DATA_DIR
% cd valid

sample_valid_imgs = glob('*.jpg')
shuffled_valid_sample = np.random.permutation(sample_valid_imgs)

for i in range(50): 
  copyfile(shuffled_valid_sample[i], DATA_DIR + '/sample/valid/' + shuffled_valid_sample[i])

/content/data
/content/data/valid


In [13]:
# Create folders for cats vs. dogs in both the full & sample datasets

% cd $DATA_DIR/sample/train
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

% cd $DATA_DIR/sample/valid
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

% cd $DATA_DIR/valid
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

% cd $DATA_DIR/train
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

/content/data/sample/train
/content/data/sample/valid
/content/data/valid
/content/data/train


In [14]:
# Test set: create an 'unknown' label for given test set because we're not sure if cat or dog

% cd $DATA_DIR/test
% mkdir unknown
% mv *.jpg unknown/

/content/data/test


In [15]:
# Start model training
# Set up where to find the datasets

% cd $DATA_DIR

# Run pipeline on sample to ensure it's working
path = DATA_DIR + '/' # '/sample/'; otherwise just + '/' 

test_path = DATA_DIR + '/test/'
results_path = DATA_DIR + '/results/'

train_path = path + '/train/'
valid_path = path + '/valid/'

/content/data


In [16]:
# Instantiate vgg16 class
vgg = Vgg16()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
40960/35363 [==================================] - 0s 2us/step


In [0]:
# Set hyperparameters
batch_size=64
num_epochs=3
#vgg.model.optimizer.lr = 0.01 

In [18]:
# Run model finetuning to switch down to only 2 classes (cats & dogs)

batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size) # batch_size=batch_size*2
vgg.finetune(batches)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [0]:
results_path = "/content/drive/'My Drive'/'Colab Notebooks'/fastai/"

In [20]:
# Run model training to fit the model
# Training: 20000 observations / 64 batch size = 312 groups

'''
for epoch in range(num_epochs):
  print("Running epoch: %d" % epoch)
  vgg.fit(batches, val_batches, nb_epoch=num_epochs)
  current_weights = 'ft%d.h5' % epoch # save weights for every pass in a .h5 file format
  vgg.model.save_weights(results_path + current_weights) # note: should switch this to persistent storage on Google Drive
print("Fit the model over " + str(num_epochs) + " epochs")
'''

vgg.fit(batches, val_batches, nb_epoch=num_epochs)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
312/312 [==============================] - 254s 813ms/step - loss: 0.4150 - acc: 0.8153 - val_loss: 0.2500 - val_acc: 0.8918
Epoch 2/3
312/312 [==============================] - 248s 794ms/step - loss: 0.3436 - acc: 0.8519 - val_loss: 0.2333 - val_acc: 0.9015
Epoch 3/3
312/312 [==============================] - 244s 781ms/step - loss: 0.3253 - acc: 0.8616 - val_loss: 0.2175 - val_acc: 0.9092


In [0]:
# Save weights to Colab instance, then cp it over to Google Drive for persistence 

tmp_results_path = '/content/data/'

current_weights = 'catdog_weights.h5'
vgg.model.save_weights(tmp_results_path + current_weights) 

In [25]:
# Check that the weights saved properly
!ls $tmp_results_path

catdog_weights.h5  results  sample_submission.csv  test.zip  train.zip
data		   sample   test		   train     valid


In [27]:
% cp $tmp_results_path/catdog_weights.h5 /content/drive/'My Drive'/'Colab Notebooks'/fastai/catdog_weights.h5

'/content/drive/My Drive/Colab Notebooks/fastai/catdog_weights.h5'


In [28]:
% ls /content/drive/'My Drive'/'Colab Notebooks'/fastai/

all.zip                  kaggle.json
catdog_weights.h5        lesson1-homework-cats-dogs-redux.ipynb
convolution-intro.ipynb  lesson1.ipynb
data/                    lesson2.ipynb
devhub-workshop.ipynb    lesson3.ipynb
dogscats.zip             sgd-intro.ipynb
